# Lesson 3: Enable Logging

### Import all needed packages

In [2]:
import boto3
import json
import os

# creating a bedrock client
bedrock = boto3.client('bedrock', region_name="us-west-2")

In [4]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [6]:
# Defining a log group name(where we want to send our log files to)
log_group_name = '/my/amazon/bedrock/logs'

In [7]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [8]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        #arn- amazon resource name
        'roleArn': os.environ['LOGGINGROLEARN'], 
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [9]:
#Applying logging configuration to bedrock.
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'f288af9a-50b7-4e7c-9c7e-eec717198dd3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 19 Mar 2024 07:30:55 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f288af9a-50b7-4e7c-9c7e-eec717198dd3'},
  'RetryAttempts': 0}}

In [10]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '31ea8092-c1a0-45ae-828f-f3951227986d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 19 Mar 2024 07:32:19 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': '31ea8092-c1a0-45ae-828f-f3951227986d'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::557714912334:role/c99355a2566044l6191762t1w55771491233-LoggingIAMRole-HhRyzt9FE1Y0',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l6191762t1w557714912-loggings3bucket-b2npockq5ayt',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l6191762t1w557714912-loggings3bucket-b2npockq5ayt',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [11]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [12]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet that has been the subject of much speculation and discussion among science fiction fans and writers. The planet is first mentioned in the 1968 science fiction novel "The Hitchhiker's Guide to the Galaxy" by Douglas Adams. In the novel, Foobar is a planet that is home to a highly advanced and sophisticated civilization that is known for its art, music, and literature. The planet is also home to a unique species of alien beings known as the Vogon, who are known for their bureaucratic and unpleasant nature.

Despite its fictional status, Foobar has become a popular symbol in the science fiction community, and has been referenced in a wide range of works since the publication of "The Hitchhiker's Guide to the Galaxy." The planet has been depicted in various forms in different media, including movies, television shows, video games, and comic books. In some depictions, Foobar is portrayed as a utopia, while in others, it is depicted as a dystopia.

One of the mo

In [13]:
# Note: pulls back all the entries from the last 5min
cloudwatch.print_recent_logs(log_group_name)

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [14]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [ ]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
